# 07. 주성분 분석 (PCA)

차원 축소의 가장 기본적인 방법인 PCA를 구현합니다.

## 학습 목표
- 공분산 행렬과 고유값 분해 이해
- 주성분의 의미 이해
- 차원 축소 및 데이터 시각화

In [ ]:
import torch
import matplotlib.pyplot as plt

torch.manual_seed(42)

## 1. PCA 원리

데이터의 분산을 최대화하는 방향(주성분)을 찾습니다.

In [ ]:
# 2D 상관 데이터 생성
n_samples = 200
X = torch.randn(n_samples, 2)
# 상관관계 추가
X[:, 1] = X[:, 0] * 0.8 + X[:, 1] * 0.3

plt.figure(figsize=(6, 6))
plt.scatter(X[:, 0], X[:, 1], alpha=0.7)
plt.xlabel('x1')
plt.ylabel('x2')
plt.title('Correlated 2D Data')
plt.axis('equal')
plt.grid(True, alpha=0.3)
plt.show()

In [ ]:
from mlfs.classical.reduction import PCA

In [ ]:
# PCA 적용
pca = PCA(n_components=2)
pca.fit(X)

# 주성분 시각화
mean = pca.mean_
components = pca.components_
explained_var = pca.explained_variance_

print(f"Explained Variance: {explained_var}")
print(f"Explained Variance Ratio: {explained_var / explained_var.sum()}")

plt.figure(figsize=(6, 6))
plt.scatter(X[:, 0], X[:, 1], alpha=0.5)

# 주성분 화살표
for i in range(2):
    scale = torch.sqrt(explained_var[i]) * 2
    plt.arrow(mean[0], mean[1], 
              components[i, 0] * scale, components[i, 1] * scale,
              head_width=0.1, head_length=0.05, fc=f'C{i}', ec=f'C{i}',
              label=f'PC{i+1}')

plt.xlabel('x1')
plt.ylabel('x2')
plt.title('Principal Components')
plt.legend()
plt.axis('equal')
plt.grid(True, alpha=0.3)
plt.show()

## 2. MNIST 차원 축소

In [ ]:
from mlfs.utils.data import load_mnist_subset

X_mnist, y_mnist = load_mnist_subset(n_samples=2000, flatten=True)
print(f"Original shape: {X_mnist.shape}")

In [ ]:
# 2D로 축소
pca_mnist = PCA(n_components=2)
X_reduced = pca_mnist.fit_transform(X_mnist)

print(f"Reduced shape: {X_reduced.shape}")

# 시각화
plt.figure(figsize=(10, 8))
scatter = plt.scatter(X_reduced[:, 0], X_reduced[:, 1], 
                      c=y_mnist, cmap='tab10', alpha=0.7, s=10)
plt.colorbar(scatter, label='Digit')
plt.xlabel('PC1')
plt.ylabel('PC2')
plt.title('MNIST - PCA (2D)')
plt.show()

In [ ]:
# 다양한 차원에서의 설명력
pca_full = PCA(n_components=100)
pca_full.fit(X_mnist)

explained_ratio = pca_full.explained_variance_ / pca_full.explained_variance_.sum()
cumulative_ratio = torch.cumsum(explained_ratio, dim=0)

plt.figure(figsize=(10, 4))

plt.subplot(1, 2, 1)
plt.bar(range(1, 21), explained_ratio[:20].numpy())
plt.xlabel('Principal Component')
plt.ylabel('Explained Variance Ratio')
plt.title('Individual Explained Variance')

plt.subplot(1, 2, 2)
plt.plot(range(1, 101), cumulative_ratio.numpy())
plt.axhline(y=0.95, color='r', linestyle='--', label='95%')
plt.xlabel('Number of Components')
plt.ylabel('Cumulative Explained Variance')
plt.title('Cumulative Explained Variance')
plt.legend()
plt.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

# 95% 설명력을 위한 성분 수
n_95 = (cumulative_ratio >= 0.95).nonzero()[0, 0].item() + 1
print(f"\n95% variance explained with {n_95} components")

## 3. 재구성

축소된 데이터에서 원본 복원

In [ ]:
from mlfs.utils.viz import plot_images
from mlfs.utils.data import load_mnist

# 원본 이미지
X_img, _ = load_mnist(train=True, flatten=False)

# 다양한 차원으로 축소 후 재구성
n_components_list = [10, 50, 100, 200]

fig, axes = plt.subplots(len(n_components_list) + 1, 5, figsize=(10, 10))

# 원본
for i in range(5):
    axes[0, i].imshow(X_img[i, 0], cmap='gray')
    axes[0, i].axis('off')
axes[0, 0].set_ylabel('Original', fontsize=12)

# 재구성
X_flat = X_mnist[:5]
for row, n_comp in enumerate(n_components_list, 1):
    pca = PCA(n_components=n_comp)
    X_reduced = pca.fit_transform(X_flat)
    X_reconstructed = pca.inverse_transform(X_reduced)
    
    for i in range(5):
        img = X_reconstructed[i].reshape(28, 28)
        axes[row, i].imshow(img, cmap='gray')
        axes[row, i].axis('off')
    axes[row, 0].set_ylabel(f'n={n_comp}', fontsize=12)

plt.suptitle('PCA Reconstruction', fontsize=14)
plt.tight_layout()
plt.show()

## 요약

1. **PCA**: 분산을 최대화하는 방향 찾기
2. **고유값 분해**: 공분산 행렬의 고유벡터 = 주성분
3. **차원 축소**: 중요한 성분만 유지
4. **시각화**: 고차원 데이터를 2D/3D로 표현

다음: 비선형 차원축소인 **LLE**와 **Isomap**